In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

file_path = "../../data/processed/medical_equipment_utilization_synthetic_cleaned(in).csv"

raw_df = pd.read_csv(file_path)
raw_df.columns = [c.strip().replace(" ", "_") for c in raw_df.columns]

df = raw_df.dropna(axis=1, how="all").copy()

currency_cols = [
    "Cost_per_procedure",
    "Daily_Operating_Cost",
    "Procedure_Revenue",
    "Net-Profit_(daily)",
]
for col in currency_cols:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace("KES", "", case=False, regex=False)
            .str.replace(",", "", regex=False)
            .str.replace(" ", "", regex=False)
        )
        df[col] = pd.to_numeric(df[col], errors="coerce")

numeric_cols = [
    "Available_Hours",
    "Actual_Operating_Hours",
    "Utilization_Rate",
    "Number_of_Procedures",
]
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

if "Date" in df.columns:
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

print(f"Loaded {len(df)} rows")


In [ ]:
fig_util = px.histogram(
    df,
    x="Utilization_Rate",
    nbins=30,
    marginal="box",
    title="Utilization Rate Distribution",
)
fig_util.update_layout(bargap=0.05)
fig_util.show()


In [ ]:
fig_hours = px.scatter(
    df,
    x="Available_Hours",
    y="Actual_Operating_Hours",
    color="Equipment_Type",
    hover_data=["Equipment_ID", "Department"],
    title="Available vs Actual Operating Hours",
)
fig_hours.show()


In [ ]:
by_eq = df["Equipment_Type"].value_counts().reset_index()
by_eq.columns = ["Equipment_Type", "Count"]
fig_eq = px.bar(by_eq, x="Equipment_Type", y="Count", title="Records by Equipment Type")
fig_eq.update_layout(xaxis_tickangle=-45)
fig_eq.show()


In [ ]:
if "Department" in df.columns:
    by_dept = df["Department"].value_counts().reset_index()
    by_dept.columns = ["Department", "Count"]
    fig_dept = px.bar(by_dept, x="Department", y="Count", title="Records by Department")
    fig_dept.update_layout(xaxis_tickangle=-45)
    fig_dept.show()
